In [ ]:
import pandas as pd
import hvplot.pandas
import numpy as np
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")

df = pd.read_csv(snakemake.input["lcoes"])

df = df.drop(columns=["capacities","class"])

df["cumulative generation"] /= 1e6 # in TWh
df["annual generation"] /= 1e6 # in TWh
df = df.sort_values("lcoe")

df["tech cum gen"] = df.groupby("technology")["annual generation"].cumsum()

plot = (
    df.hvplot.step(x="tech cum gen", y="lcoe", by="technology", xlabel="Technology potential [TWh]", ylabel="LCoE [EUR/MWh]", title="Technology-specific supply", legend='top_left')
    + df.hvplot.area(x="cumulative generation",y="lcoe", by="technology", stacked=False, ylabel="LCoE [EUR/MWh]", xlabel="Total potential [TWh]", title="Combined supply", legend='top_left', yaxis="right")
    * df[df["reserved share"] > 0].hvplot.area(x="cumulative generation", y="lcoe", fill_color=None, c="k", line_dash="dotted")
)

for of in snakemake.output["fig"]:
    hvplot.save(plot, of)